In [14]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import urllib.parse
import ssl
import requests


# SSL 에러 방지
context = ssl._create_unverified_context()

# 기존 데이터셋 로드
data = pd.read_csv('Geumcheon_new.csv', encoding='utf-8-sig')

# 숙박 정보를 담을 새로운 열 추가
data['숙박정보'] = ''

# 관광지명만 추출
tourist_spots = data['관광지명'].tolist()

# 관광지명을 URL에 적합하도록 인코딩
encoded_list = [urllib.parse.quote(s) for s in tourist_spots]

# 숙박 정보 가져오기
for spot, row_index in zip(encoded_list, range(len(encoded_list))):
    basic = f'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query={spot}+%EA%B7%BC%EC%B2%98%20%EC%88%99%EB%B0%95'
    url = requests.get(basic, headers={'User-Agent':'Mozilla/5.0'})
    html = urlopen(basic, context=context).read()
    soup = BeautifulSoup(html, 'html.parser')

    # 숙박시설명 + 카테고리 추출
    span_name = soup.find_all(class_='YwYLL')
    span_category = soup.find_all(class_='YzBgS')

    # 각 숙박시설명과 카테고리를 묶어서 맛집 정보 생성
    accommodation_info = []
    for name, category in zip(span_name, span_category):
        accommodation_info.append((name.text.strip(), category.text.strip()))

    # 데이터프레임에 숙박 정보 저장
    data.at[row_index, '숙박정보'] = accommodation_info

# 결과 출력
filtered_data = data[data['숙박정보'].apply(lambda x: len(x) > 0)]
filtered_data.head()

,시군구명,시도명,관광지명,Unnamed: 3,KTO카테고리대분류명,KTO카테고리중분류명,KTO카테고리소분류명,방문자수,맛집정보,숙박정보
0,금천구,서울특별시,독산동우시장,https://korean.visitkorea.or.kr/detail/ms_deta...,쇼핑,쇼핑,상설시장,893,"[('경성양꼬치 구로디지털직영점', '양꼬치'), ('로바다야끼소담', '이자카야'...","[(88호텔, 호텔), (3S호텔, 호텔), (호텔인카페 독산사거리점, 호텔), (..."
1,금천구,서울특별시,마리오아울렛,https://korean.visitkorea.or.kr/detail/ms_deta...,쇼핑,쇼핑,전문매장/상가,285,"[('경성양꼬치 구로디지털직영점', '양꼬치'), ('돼지네 본점', '돼지고기구이...","[(신라스테이 구로, 호텔), (포포인츠 바이 쉐라톤 서울 구로, 호텔), (롯데시..."
2,금천구,서울특별시,비단길 현대시장,https://korean.visitkorea.or.kr/detail/ms_deta...,쇼핑,쇼핑,상설시장,214,"[('닭헌터', '한식'), ('한양마늘보쌈', '족발,보쌈'), ('도마다리뼈다귀...","[(엠디호텔 독산, 호텔), (호텔 트리플 8, 호텔), (3S호텔, 호텔), (타..."
4,금천구,서울특별시,별빛 남문시장,https://korean.visitkorea.or.kr/detail/ms_deta...,쇼핑,쇼핑,상설시장,183,"[('국민삼오통닭', '치킨,닭강정')]","[(대림장여관, 여관), (신라스테이 구로, 호텔), (포포인츠 바이 쉐라톤 서울 ..."
5,금천구,서울특별시,산기슭 공원,https://korean.visitkorea.or.kr/detail/ms_deta...,자연,자연관광지,자연생태관광지,153,"[('동흥관', '중식당'), ('리화진 금천 독산본점', '요리주점'), ('빅터...","[(엠디호텔 독산, 호텔), (3S호텔, 호텔), (호텔 트리플 8, 호텔), (독..."


In [15]:
# 데이터프레임을 새로운 CSV 파일로 저장
filtered_data.to_csv('Geumcheon_all.csv', index=False, encoding='utf-8-sig')